In [1]:
## Import library gb_mep
import gb_mep

In [2]:
## Load all required files after importing relevant libraries
import pickle
import numpy as np
import pandas as pd
santander_train = pd.read_csv('../data/santander_train.csv') 
santander_test = pd.read_csv('../data/santander_test.csv') 
santander_distances = np.load('../data/santander_distances.npy')
with open('../data/santander_dictionary.pkl', 'rb') as f:
    santander_dictionary = pickle.load(f)


In [3]:
santander_train

,start_id,end_id,start_time,end_time
0,722,131,0.483492,13.192747
1,531,460,0.484768,23.477447
2,515,603,0.624303,4.196019
3,804,689,0.807900,13.604670
4,740,48,1.568787,31.912915
...,...,...,...,...
6453258,302,637,241912.321072,241919.498489
6453259,740,342,241912.686823,241919.489431
6453260,327,324,241913.941339,241917.596443
6453261,224,337,241913.962367,241919.498538


In [4]:
G = gb_mep.gb_mep(df=santander_train, id_map=santander_dictionary, distance_matrix=santander_distances)


In [5]:
node = 1
self = G
thresh = 0
from scipy.optimize import minimize
print('\r', node, '-', self.id_map[node], ' '*20, end='\r')
# Recursion
print('\r', node, '-', self.id_map[node], ' '*20, end='\r')
ds = self.distance_matrix[node]
# Recursion
time_diffs_A = {}; time_diffs_A_prime = {}
for secondary_node in np.where(ds <= thresh)[0]:
    start_breaks = np.searchsorted(a=self.start_times[secondary_node], v=self.start_times[node], side='left')
    end_breaks = np.searchsorted(a=self.end_times[secondary_node], v=self.start_times[node], side='left')
    for k, t in enumerate(self.start_times[node]):
        if k != 0:
            time_diffs_A[k, secondary_node] = t - self.start_times[secondary_node][start_breaks[k-1]:start_breaks[k]]
            time_diffs_A_prime[k, secondary_node] = t - self.end_times[secondary_node][end_breaks[k-1]:end_breaks[k]]
        else:
            time_diffs_A[k, secondary_node] = t - self.start_times[secondary_node][:start_breaks[k]]
            time_diffs_A_prime[k, secondary_node] = t - self.end_times[secondary_node][:end_breaks[k]]
res = minimize(fun=self.negative_loglikelihood_full, x0=np.array([np.log((k+1)/self.T), -1.0, -2.0, -1.0, -1.0, -2.0, 0.0]),
                        args=(node, time_diffs_A, time_diffs_A_prime, np.where(ds <= thresh)[0]))

[ 0.01347553  0.36787944 -0.86466472  0.36787944  0.36787944 -0.86466472
  1.        ]
[ 0.01347553  0.36787944 -0.86466472  0.36787944  0.36787944 -0.86466472
  1.        ]
[ 0.01347553  0.36787945 -0.8646647   0.36787944  0.36787944 -0.86466472
  1.        ]
[ 0.01347553  0.36787944 -0.86466471  0.36787944  0.36787944 -0.86466472
  1.        ]
[ 0.01347553  0.36787944 -0.86466472  0.36787945  0.36787944 -0.86466472
  1.        ]
[ 0.01347553  0.36787944 -0.86466472  0.36787944  0.36787945 -0.8646647
  1.        ]
[ 0.01347553  0.36787944 -0.86466472  0.36787944  0.36787944 -0.86466471
  1.        ]


/Users/fraspass/Documents/github/gb-mep/lib/gb_mep/gbmep.py:75: RuntimeWarning: overflow encountered in exp
  ll += np.exp(-params[3] * ds[node]) * params[1] / params[2] * np.sum(np.exp(-params[2] * (self.T - self.start_times[node])) - 1)
/Users/fraspass/Documents/github/gb-mep/lib/gb_mep/gbmep.py:76: RuntimeWarning: overflow encountered in exp
  ll += np.exp(-params[6] * ds[node]) * params[4] / params[5] * np.sum(np.exp(-params[5] * (self.T - self.end_times[node])) - 1)
/Users/fraspass/Documents/github/gb-mep/lib/gb_mep/gbmep.py:79: RuntimeWarning: overflow encountered in scalar multiply
  A[k, node] = ((np.exp(-params[2] * time_diffs[k-1]) * A[k-1, node]) if k > 0 else 0) + np.sum(np.exp(-params[2] * time_diffs_A[k, node]))
/Users/fraspass/Documents/github/gb-mep/lib/gb_mep/gbmep.py:80: RuntimeWarning: overflow encountered in scalar multiply
  A_prime[k, node] = ((np.exp(-params[5] * time_diffs[k-1]) * A_prime[k-1, node]) if k > 0 else 0) + np.sum(np.exp(-params[5] * time_diffs_A_pri

[ 0.01347553  0.36787944 -0.86466472  0.36787944  0.36787944 -0.86466472
  1.00000001]


In [11]:
np.sum(np.exp(-.000000000000000000000000000000001 * (self.T - self.start_times[node])) - 1)

0.0

In [12]:
np.exp(-4*0)

1.0

In [18]:
node_index = 1
ds = self.distance_matrix[node_index]

In [11]:
np.ones((10,5)).sum(axis=1)

array([5., 5., 5., 5., 5., 5., 5., 5., 5., 5.])